In [4]:
from manim import *
import numpy as np


config.media_width = "75%"
config.verbosity = "WARNING"

In [7]:
%%manim -qm NuclearScatteringAsym2

class NuclearScatteringAsym2(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create axes
        axes = ThreeDAxes(
            x_range=[-4, 4],
            y_range=[-4, 4],
            z_range=[-4, 4],
        )
        axes.set_opacity(0.2)

        # Create nucleus
        nucleus = Sphere(radius=0.2, color=GRAY)

        # Create particle at (0, -1, 0)
        particle = Sphere(radius=0.1, color=RED)
        particle.move_to([0, -1, 0])

        # Orbit path in the YZ plane
        orbit_path = Circle(radius=1, color=GRAY).set_opacity(0.5)
        orbit_path.rotate(90 * DEGREES, axis=UP)

        # Radial force vector (scaled r-vector)
        radial_force = Arrow(
            start=ORIGIN,
            end=particle.get_center(),
            color=PURPLE,
            buff=0
        )

        # p × S vector (constant in Y direction)
        cross_product = Arrow(
            start=particle.get_center(),
            end=particle.get_center() + np.array([0, 1, 0]),
            color=ORANGE,
            buff=0
        )

        # Combined force vector
        total_force = Arrow(
            start=particle.get_center(),
            end=particle.get_center() + np.array([0, 1, 0]),
            color=WHITE,
            buff=0
        )

        # Updater for radial force vector
        def update_radial_force(vector):
            r = particle.get_center()
            r_magnitude = np.linalg.norm(r)
            r_hat = r / r_magnitude if r_magnitude > 0 else np.array([0, 0, 0])
            
            S_cross_p = np.array([0, 1, 0])  # This is p × S
            f_radial = np.dot(r, S_cross_p)
            
            force_vector = f_radial * r_hat
            vector.put_start_and_end_on(ORIGIN, force_vector)
            return force_vector  # Return for use in total force calculation

        # Updater for cross product vector
        def update_cross_product(vector):
            pos = particle.get_center()
            S_cross_p = np.array([0, 1, 0])
            vector.put_start_and_end_on(pos, pos + S_cross_p)
            return S_cross_p  # Return for use in total force calculation

        # Updater for total force vector
        def update_total_force(vector):
            pos = particle.get_center()
            # Get the current radial force and cross product vectors
            r = pos
            r_magnitude = np.linalg.norm(r)
            r_hat = r / r_magnitude if r_magnitude > 0 else np.array([0, 0, 0])
            
            S_cross_p = np.array([0, 1, 0])
            f_radial = np.dot(r, S_cross_p)
            radial_component = f_radial * r_hat
            
            # Calculate total force (sum of components)
            total = radial_component + S_cross_p
            
            # Update the arrow
            vector.put_start_and_end_on(pos, pos + total)

        radial_force.add_updater(update_radial_force)
        cross_product.add_updater(update_cross_product)
        total_force.add_updater(update_total_force)

        # Add everything to scene
        self.add(nucleus, particle, orbit_path, radial_force, cross_product, total_force, axes)

        # Create the fixed label on top
        label_text = MathTex("F_{radial} = ", "\\vec{r}", "\\cdot", 
            "\\big(", "\\vec{p}", "\\times", "\\vec{S}", "\\big)", color=WHITE)
        label_text.to_corner(UR).shift(LEFT * 2)
        self.add_fixed_in_frame_mobjects(label_text)
        label_text[1].set_color(PURPLE)  # r

        # Add cross product label
        cross_label = MathTex("\\vec{p}", "\\times", "\\vec{S}", color=ORANGE)
        cross_label.to_corner(UL).shift(RIGHT * 1 + DOWN * 0.5)
        self.add_fixed_in_frame_mobjects(cross_label)

        # Add total force label
        total_label = MathTex("\\vec{F}_{total}", color=WHITE)
        total_label.next_to(cross_label, DOWN)
        self.add_fixed_in_frame_mobjects(total_label)

        # Create the updating value text
        display_text = MathTex("00.00", color=WHITE)
        display_text.next_to(label_text, RIGHT, buff=0.32)
        self.add_fixed_in_frame_mobjects(display_text)

        # Update just the value text
        def update_text(mob):
            r = particle.get_center()
            S_cross_p = np.array([0, 1, 0])
            f_radial = np.dot(r, S_cross_p)
            new_text = MathTex(f"{f_radial:.2f}", color=WHITE)
            new_text.move_to(mob)
            mob.become(new_text)

        display_text.add_updater(update_text)

        # # Begin ambient camera rotation
        # self.begin_ambient_camera_rotation(rate=0.9)
        
        for _ in range(3):
            # Move particle arou
            self.play(
                Rotating(
                    particle,
                    about_point=ORIGIN,
                    axis=RIGHT,
                    angle=TAU,  # Rotate 3 full times
                    run_time=4,  # Total time for all rotations
                    rate_func=linear
                )
            )

        self.wait(2)

        # Clean up updaters
        radial_force.clear_updaters()
        cross_product.clear_updaters()
        total_force.clear_updaters()
        display_text.clear_updaters()

Manim Community v0.18.1

In [93]:
%%manim -qm NucleonForce

class NucleonForce(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create nucleon at center
        nucleon = Sphere(radius=0.3, color=GRAY)
        
        # Create main coordinate axes
        axes = ThreeDAxes(
            x_range=[-2, 2, 1],
            y_range=[-2, 2, 1],
            z_range=[-2, 2, 1],
            axis_config={"include_tip": True, "tip_width": 0.2, "tip_height": 0.2}
        ).set_opacity(0.8)

        # Add labels with nuclear physics convention
        z_label = MathTex("z").next_to(axes.z_axis.get_end(), RIGHT).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        y_label = MathTex("y").next_to(axes.y_axis.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        x_label = MathTex("x").next_to(axes.x_axis.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Create phi arc (azimuthal angle)
        phi_arc = always_redraw(
            lambda: Arc(
                radius=2,
                angle=2 * PI,  # Example angle
                color=GREEN
            ).rotate(90 * DEGREES, axis=UP)
        )

        # Create theta arc (polar angle) should go from x to z axis, 90 degrees at radius 1
        theta_arc = always_redraw(
            lambda: Arc(
                radius=1,
                angle=1/2 *PI,  # Example angle
                color=BLUE
            ).rotate(90 * DEGREES, axis=RIGHT).move_to([0.5, 0,0.5])
        )

        # Add angle labels
        theta_label = MathTex("\\theta").move_to([0.9, 0, 0.9]).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        theta_label.color = BLUE
        phi_label = MathTex("\\phi").move_to([0, 1.6, 1.6]).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        phi_label.color = GREEN

        # Add everything to the scene
        self.add(axes, y_label, z_label, x_label)
        self.play(Create(theta_arc), Write(theta_label), run_time=3)
        self.play(Write(phi_label), Create(phi_arc), run_time=3)

        self.begin_ambient_camera_rotation(rate=0.5)
        self.wait(3)
        



Manim Community v0.18.1

In [117]:
%%manim -qm NucleonForce

class NucleonForce(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Create nucleon at center
        nucleon = Sphere(radius=0.3, color=GRAY)
        
        # Create main coordinate axes
        axes = ThreeDAxes(
            x_range=[-2, 2, 1],
            y_range=[-2, 2, 1],
            z_range=[-2, 2, 1],
            axis_config={"include_tip": True, "tip_width": 0.2, "tip_height": 0.2}
        ).set_opacity(0.8)

        # Add labels with nuclear physics convention
        z_label = MathTex("z").next_to(axes.z_axis.get_end(), RIGHT).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        y_label = MathTex("y").next_to(axes.y_axis.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        x_label = MathTex("x").next_to(axes.x_axis.get_end(), UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)

        # Create momentum equations (fixed in frame)
        initial_momentum = MathTex(
            "\\vec{p} = (p_0, 0, 0)",
            color=WHITE
        ).to_corner(UL).shift(0.1*RIGHT + 0.1*DOWN).set_opacity(0)
        self.add_fixed_in_frame_mobjects(initial_momentum)

        final_momentum = MathTex(
            "\\vec{p}\\,' = (p' \\sin", "\\vartheta", "\\cos", " \\phi,", 
            "p' \\sin"," \\vartheta","\\sin ",",\\phi,",
            "p' \\cos",  "\\vartheta)",
            color=WHITE
        ).to_corner(DL).shift(0.1 * RIGHT + 0.1* UP).set_opacity(0)
    
        self.add_fixed_in_frame_mobjects(final_momentum)
        final_momentum[1].set_color(BLUE)
        final_momentum[3].set_color(GREEN)
        final_momentum[5].set_color(BLUE)
        final_momentum[7].set_color(GREEN)
        final_momentum[9].set_color(BLUE)

        # Create phi arc (azimuthal angle)
        phi_arc = always_redraw(
            lambda: Arc(
                radius=2,
                angle=2 * PI,
                color=GREEN
            ).rotate(90 * DEGREES, axis=UP)
        )

        # Create theta arc (polar angle)
        theta_arc = always_redraw(
            lambda: Arc(
                radius=1,
                angle=1/2 *PI,
                color=BLUE
            ).rotate(90 * DEGREES, axis=RIGHT).move_to([0.5, 0,0.5])
        )

        # Add angle labels
        theta_label = MathTex("\\theta").move_to([0.9, 0, 0.9]).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        theta_label.color = BLUE
        phi_label = MathTex("\\phi").move_to([0, 1.6, 1.6]).rotate(90 * DEGREES, axis=UP).rotate(90 * DEGREES, axis=RIGHT)
        phi_label.color = GREEN

        # Create incoming nucleon (small red sphere)
        incoming_nucleon = Sphere(radius=0.1, color=RED)
        incoming_nucleon.move_to([-2, 0, 0])  # Start from negative x-axis

        # Create the scattering path
        incoming_path = Line(start=[-2, 0, 0], end=[0, 0, 0], color=RED)
        
        # Create scattered path (at some angle)
        scattered_path = Line(
            start=[0, 0, 0],
            end=[1, 1, -1],  # Example scattering direction
            color=RED
        )

        # Add everything to the scene
        self.add(axes, y_label, z_label, x_label, nucleon)
        self.play(Create(theta_arc), Write(theta_label), run_time=3)
        self.play(Write(phi_label), Create(phi_arc), run_time=3)
    

        # Add scattering animation
        self.play(Create(incoming_path))
        self.play(
            incoming_nucleon.animate.move_to([-2, 0, 0]),
            run_time=0.1
        )
        self.play(
            MoveAlongPath(incoming_nucleon, incoming_path),
            Write(initial_momentum),
            initial_momentum.animate.set_opacity(1),
            run_time=2

        )
        self.play(
            Create(scattered_path),
            MoveAlongPath(incoming_nucleon, scattered_path),
            Write(final_momentum),
            final_momentum.animate.set_opacity(1),
            run_time=2
        )

        # rotate the camera to see the scattering
        self.begin_ambient_camera_rotation(rate=0.3)
        self.wait(3)
        self.stop_ambient_camera_rotation()

        self.wait(3)

Manim Community v0.18.1

In [192]:
%%manim -qm SpinScatteringAnimation

class SpinScatteringAnimation(Scene):
    def construct(self):
        # Initial formulas for p and p' with vector notation
        initial_p = MathTex(
            r"\vec{p} = (0, 0, p)"
        ).shift(UP * 2)
        
        initial_p_prime = MathTex(
            r"\vec{p}^{\,\prime} = (p'\sin\vartheta\cos\phi,",
            r"p'\sin\vartheta\sin\phi,",
            r"p'\cos\vartheta)"
        ).next_to(initial_p, DOWN)

        # Create first scene with initial formulas
        self.play(Write(initial_p))
        self.play(Write(initial_p_prime))
        self.wait(2)

        # Azimuthal distribution formula
        azimuthal_title = Text("Angular Distribution of scattered nucleons:", font_size=36).shift(UP * 2.2)
        azimuthal_formula1 = MathTex(
            r"\cos(\phi - \phi_0) =", # index 0
            r"\cos\phi\cos\phi_0 + \sin\phi\sin\phi_0" # index 1
        ).move_to([0, -2, 0])
        
        # Start with right side hidden
        azimuthal_formula1[1].set_opacity(0)

        # Animate transition to azimuthal distribution
        self.play(
            FadeOut(initial_p),
            initial_p_prime.animate.scale(0.8).shift(UP * 2),
            FadeIn(azimuthal_title),
        )
        self.wait(1)

        # Add cross product term with vector notation
        cross_product = MathTex(
            r"-\vec{p} \times \vec{S} = (pS_y, -pS_x, 0)"
        ).move_to([0, 0.5, 0])

        # Show cross product and its relationship
        self.play(Write(cross_product))
        self.wait(2)

        # Final preferred angle formulas with step-by-step transformation
        preferred_angle = MathTex(
            r"\tan\phi_0 ", 
            r"= -\frac{S_x}{S_y}"
        ).move_to([0, 0, 0])

        preferred_angle2 = MathTex(
            r"\frac{\sin\phi_0}{\cos\phi_0} ",
            r"= -\frac{S_x}{S_y}"
        ).move_to([0, 0, 0])

        self.wait(2)

        # Show initial form of preferred angle
        self.play(
            Write(preferred_angle),
            cross_product.animate.shift(UP * 0.5)
        )
        self.wait(2)

        # Reveal the right side of the equation with trigonometric identity
        self.play(Write(azimuthal_formula1[0]))
        self.wait(2)
        self.play(
            azimuthal_formula1[1].animate.set_opacity(1),
            run_time=2
        )
        self.wait(2)

        # Proportional relationship with spin components
        azimuthal_formula2 = MathTex(
            r"\cos(\phi - \phi_0) ", # index 0
            r"\propto S_y\cos\phi - S_x\sin\phi" # index 1
        ).move_to([0, -2, 0])

        self.play(Transform(preferred_angle, preferred_angle2))

        # Final transition
        self.play(Transform(azimuthal_formula1, azimuthal_formula2))
        self.wait(2)

        self.play(FadeOut(preferred_angle), FadeOut(cross_product), FadeOut(initial_p_prime),
            azimuthal_formula1.animate.move_to([0, 0, 0]))
        self.wait(5)
        
        # Final distribution formula with color coding
        final_formula = MathTex(
            r"f(p, \vartheta, \phi)", # 0
            r"=", # 1
            r"\frac{\varepsilon(p, \vartheta)}{2\pi}", # 2
            r"(1", # 3
            r"+", # 4
            r"A_y(p, \vartheta)", # 5
            r"S_y", # 6
            r"\cos\phi", # 7
            r"-", # 8
            r"A_y(p, \vartheta)", # 9
            r"S_x", # 10
            r"\sin\phi", # 11
            r")" # 12
        ).move_to([0, 0, 0])

        # Color scheme
        final_formula[0].set_color(WHITE)        # function name
        final_formula[2].set_color(BLUE)         # phase space factor
        final_formula[5].set_color(GREEN)        # first Ay
        final_formula[9].set_color(GREEN)        # second Ay
        final_formula[6].set_color(PURPLE)       # Sy term
        final_formula[10].set_color(RED)         # Sx term
        final_formula[7].set_color(YELLOW)       # cos phi
        final_formula[11].set_color(YELLOW)      # sin phi

        # Write the full formula first
        self.play(Transform(azimuthal_formula1, final_formula))
        self.wait(4)

        # Define each explanation step
        steps = [
            #(final_formula[0], "Angular Distribution"),
            (final_formula[2], "Polarimeter Efficiency"),
            (final_formula[3], "Unpolarized Distribution"),
            #([final_formula[5], final_formula[9]], "Analyzing Power"),
            ([final_formula[5], final_formula[6], final_formula[7]], "Up-Down Asymmetry"),
            ([final_formula[8], final_formula[9], final_formula[10], final_formula[11]], "Left-Right Asymmetry")
        ]

        # Step through each explanation
        current_brace = None
        current_text = None

        for parts, explanation in steps:
            if isinstance(parts, list):
                new_brace = Brace(VGroup(*parts), DOWN)
            else:
                new_brace = Brace(parts, DOWN)
            
            new_text = Text(explanation, font_size=24).next_to(new_brace, DOWN)

            if current_brace is None:
                self.play(
                    Create(new_brace),
                    Write(new_text)
                )
            else:
                self.play(
                    ReplacementTransform(current_brace, new_brace),
                    ReplacementTransform(current_text, new_text)
                )

            current_brace = new_brace
            current_text = new_text
            self.wait(1.5)

        # Final fade out
        self.play(
            FadeOut(current_brace),
            FadeOut(current_text)
        )
        self.wait(1)

Manim Community v0.18.1

In [189]:
%%manim -qm SpinScatteringAnimation

class SpinScatteringAnimation(Scene):
    def construct(self):
        # Final distribution formula with color coding
        final_formula = MathTex(
            r"f(p, \vartheta, \phi)", # 0
            r"=", # 1
            r"\frac{\varepsilon(p, \vartheta)}{2\pi}", # 2
            r"(1", # 3
            r"+", # 4
            r"A_y(p, \vartheta)", # 5
            r"S_y", # 6
            r"\cos\phi", # 7
            r"-", # 8
            r"A_y(p, \vartheta)", # 9
            r"S_x", # 10
            r"\sin\phi", # 11
            r")" # 12
        ).shift(UP)

        # Color scheme
        final_formula[0].set_color(WHITE)        # function name
        final_formula[2].set_color(BLUE)         # phase space factor
        final_formula[5].set_color(GREEN)        # first Ay
        final_formula[9].set_color(GREEN)        # second Ay
        final_formula[6].set_color(PURPLE)       # Sy term
        final_formula[10].set_color(RED)         # Sx term
        final_formula[7].set_color(YELLOW)       # cos phi
        final_formula[11].set_color(YELLOW)      # sin phi

        # Write the full formula first
        self.play(Write(final_formula))
        self.wait(1)

        # Define each explanation step
        steps = [
            #(final_formula[0], "Angular Distribution"),
            (final_formula[2], "Polarimeter Efficiency"),
            (final_formula[3], "Unpolarized Distribution"),
            #([final_formula[5], final_formula[9]], "Analyzing Power"),
            ([final_formula[5], final_formula[6], final_formula[7]], "Up-Down Asymmetry"),
            ([final_formula[8], final_formula[9], final_formula[10], final_formula[11]], "Left-Right Asymmetry")
        ]

        # Step through each explanation
        current_brace = None
        current_text = None

        for parts, explanation in steps:
            if isinstance(parts, list):
                new_brace = Brace(VGroup(*parts), DOWN)
            else:
                new_brace = Brace(parts, DOWN)
            
            new_text = Text(explanation, font_size=24).next_to(new_brace, DOWN)

            if current_brace is None:
                self.play(
                    Create(new_brace),
                    Write(new_text)
                )
            else:
                self.play(
                    ReplacementTransform(current_brace, new_brace),
                    ReplacementTransform(current_text, new_text)
                )

            current_brace = new_brace
            current_text = new_text
            self.wait(1.5)

        # Final fade out
        self.play(
            FadeOut(current_brace),
            FadeOut(current_text)
        )
        self.wait(1)

Manim Community v0.18.1